<a href="https://colab.research.google.com/github/abhik-99/MFSGC/blob/master/(Test)Multi_Filter_Supervised_Gene_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(Test)Mulit-filtering Supeprvised Gene Clustering
**Please Execute the Dataset Splitter notebook before coming ot this notebook.**
Upload the *DATASET_test.csv* file obtained from Dataset splitter in this notebook.

In [ ]:
!pip install -U -q PyDrive
!pip install skfeature-chappers

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut

from scipy.sparse import *

In [ ]:
#construction of ReliefF function

"""
Given a dataset, number of random instances to pick form the dataset and
number of features to consider in each iteration (k), the function returns the weigths of the attributes
of the dataset.
These weigths can then be used as the final results out of the ReliefF algorithm

Paper-

Marko Robnik-ˇSikonja and Igor Kononenko. Theoretical and empirical analysis of relieff
and rrelieff. Machine learning, 53(1-2):23–69, 2003.

"""

def hit_miss_calculator(target,instance,k=10,hit=True,c=None):
    m=len(target)
    upper,lower=instance-1,instance+1
    hits=[]
    hit_flag=False
    #finds k nearest hits
    while(not hit_flag):
      #print(upper,lower)
      if(len(hits)>=k):
        hit_flag = True
        break
      if upper < 0 and lower > m:
        hit_flag = True
        break
      if(upper>=0):
        if((target[upper]==target[instance]) and hit):
          hits.append(upper)
        elif((target[upper]!=target[instance]) and (not hit) and target[upper]==c):
          hits.append(upper)
        upper-=1          
      if(lower<m):
        if((target[lower]==target[instance]) and hit):
          hits.append(lower)
        elif((target[lower]!=target[instance]) and (not hit) and target[lower]==c):
          hits.append(lower)
        lower+=1
    hits.sort()
    return hits


def reliefF(feature,target,k=10,repetitions=10):
  if len(feature.shape)>1:
    m,n=feature.shape
  else:
    m=len(feature)
    n=1
  #print(m,n)
  observations=list(range(m))
  classes=np.unique(target)
  weights=np.zeros(n)
  d=(np.max(feature,axis=0)-np.min(feature,axis=0))*m*k

  for i in range(repetitions):
    instance=np.random.choice(observations,1)[0]
    #print("Iteration",i)
    #print(instance)
    hits=hit_miss_calculator(target,instance,k)
    hit_class_prob=len(np.where(target==target[instance])[0])/m
    #print("\nHit Probability -",hit_class_prob)
    #print("Repetition",i,"Class",target[instance],"Hits -",hits)

    miss={}
    miss_class_prob={}

    for each_class in classes:
      if(each_class != target[instance]):
        miss[each_class]=hit_miss_calculator(target,instance,k,False,each_class)
        class_prob=len(np.where(target==each_class)[0])/m
        #print(each_class,class_prob)
        miss_class_prob[each_class]=hit_class_prob/(1 - (class_prob))

    #print("Repetition",i,"Miss-",miss,"Miss Class Probability -",miss_class_prob)
    
    for hit in hits:
      if len(feature.shape)>1:
        weights-=np.subtract(feature.iloc[instance,:],feature.iloc[hit,:])/d
      else:
        weights-=np.subtract(feature.iloc[instance],feature.iloc[hit])/d
    for each_class in miss:
      for each_miss in miss[each_class]:
        if len(feature.shape)>1:
          weights+=(np.subtract(feature.iloc[instance,:],feature.iloc[each_miss,:])/d)*miss_class_prob[each_class]
        else:
          weights+=(np.subtract(feature.iloc[instance],feature.iloc[each_miss])/d)*miss_class_prob[each_class]
    
    
  return weights.tolist()

In [ ]:
#This function discretizes the given features into 3 categories
def discretize_feature(feature):
  
  mean=np.mean(feature)
  std=np.std(feature)
  discretized=np.copy(feature)
  
  discretized[np.where(feature<(mean+std/2)) ,]=2#within 1/2 std div
  discretized[np.where(feature>(mean-std/2)),]=2#within 1/2 std div
  
  discretized[np.where(feature>(mean+std/2)),]=0#greater than half
  discretized[np.where(feature<(mean-std/2)),]=1#less than half
  
  return discretized

def Xfreq(x):
  xL={}
  for e in x:
    if e not in xL:
      xL[e]=0
    else:
      xL[e]+=1
  for e in xL:
    xL[e]/=len(x)
  return xL

def XYfreq(x,y):
  freq={}
  
  rX=np.unique(x)
  rY=np.unique(y)
      
  for e in rX:
    for f in rY:
      freq[(e,f)]=round(len(np.where(y[np.where(x==e)[0]]==f)[0])/len(x),4)
       
  return freq

def mutual_info(x,y):

  xFreq=Xfreq(x)
  yFreq=Xfreq(y)
  joint=XYfreq(x,y)
  
  Xentropy=0
  for e in xFreq:
    if xFreq[e]!=0:
      Xentropy-=xFreq[e]*np.log2(xFreq[e])
      
  Yentropy=0
  for e in yFreq:
    if yFreq[e]!=0:
      Yentropy-=yFreq[e]*np.log2(yFreq[e])
      
  jentropy=0
  for e in xFreq:
    for f in yFreq:
      if joint[(e,f)]!=0:
        jentropy-=joint[(e,f)]*np.log2(joint[(e,f)])
  
  return (Xentropy+Yentropy-jentropy)

def mutual_info_wrapper(features,target):

  mi=np.array([])
  for x in features:
    discrete=discretize_feature(features[x])
    mi=np.append(mi,mutual_info(discrete,target))
  return np.array(mi)

In [ ]:
"""
This cell is used for defining the method for calculating the t-scores
"""

def t_test(df,target):
  """
  Input:
  df= Dataframe of features (n_samples,n_features)
  target= Pandas Series/1D Numpy Array containing the class labels (n_samples)
  
  Output:
  scores= Descendingly Sorted array of features based on t-test 
  """
  import numpy as np
  from scipy.stats import ttest_ind
  scores=ttest_ind(df[:][target==0],df[:][target==1])[0] #Storing just the t-test scores and discarding the p-values from the result.
  
  # scores=np.argsort(scores,0)
  return [scores] if type(scores) != np.ndarray else scores

  

In [ ]:
from scipy.sparse import *
def fisher_score(X, y):
    import numpy as np
    
    from skfeature.utility.construct_W import construct_W
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature

    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)


In [ ]:

#Pearson corelation
def pearson_corr(feature,targetClass):
  import numpy as np
  coef=[np.abs(np.corrcoef(feature[i].values,targetClass)[0,1]) for i in feature.columns]
  # range(feature.shape[1])
  coef=[0 if np.isnan(i) else i for i in coef]
  return coef


In [ ]:
#signal to noise ratio
#using weighted one-vs-all strategy for multi-class data
def signaltonoise(feature, target, axis = 0, ddof = 0):
  import numpy as np
  classes = np.unique(target)
  if len(feature.shape)<2:
    feature = feature.reshape(-1,1)
  row, _ = feature.shape
  if len(classes) <= 2:
    m = None
    std = 0
    for each in classes:
      idx = np.where(target == each)[0]
      #convinient way of doing m1-m2
      if m is None:
        m = feature.iloc[idx, :].mean(axis)
      else:
        m -= feature.iloc[idx, :].mean(axis)

      #sd1+sd2
      std += feature.iloc[idx, :].std(axis = axis, ddof = ddof)

    return np.asanyarray(m/std)

  else:
    snr_scores = [] #for storing the weighted scores
    #using the one vs all strategy for each class with
    for each in classes:
      idx = np.where(target == each)[0]
      idxn = np.where(target != each)[0]
      m = feature.iloc[idx, :].mean(axis) - feature.iloc[idxn, :].mean(axis)
      std = feature.iloc[idx, :].std(axis = axis, ddof = ddof) + feature.iloc[idxn, :].std(axis = axis, ddof = ddof) 
      snr_scores.append((m/std) * len(idx)/row) #weighted snr

    return np.asanyarray(snr_scores).sum(axis = axis)

In [ ]:
def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [ ]:
"""
Takes a Gene Augmentation Sequence and then creates Augnmented Gene Clusters and their Representatives from it.
Must be supplied with the sequence at the beginning of the Notebook. To create a Gene Sequence, run the Notebook
for "Train". There the sequences are created and downloaded.
"""
def get_augmented(features,target, cluster_sequence):
  clusters=pd.DataFrame()
  for each in cluster_sequence:
    base_gene = features[int(each)]
    for each_gene in cluster_sequence[each]:
      operation = each_gene[-1]
      gene_key = int(each_gene[:-1])
      gene = features[gene_key]
      if operation ==  "+":
        base_gene = np.add(base_gene, gene, dtype = 'float64')
      elif operation == '-':
        base_gene = np.subtract(base_gene, gene, dype = 'float64')
    clusters[int(each)] = base_gene
  print("Clusters made. Returning Clusters")
  return clusters
#return value is similar to get_cluster in Train Notebook to make sure compatibility with evaluation code from it.

In [ ]:
files.upload()

In [ ]:
#DATASET is the name of the dataset being used.
DATASET="SRBCT"

#NEIGHBOURS determines neighbours arg for ReliefF
#for any dataset which contains any class sample 
# <10, make it less than 10. Eg of such dataset - SRBCT
NEIGHBOURS = 2 

#p is the number of top genes taken after sorting the filter scores
p=600

#q is the number of top augmented genes chosen from each filter after running 
#SGC
q = 5

data_df=pd.read_csv("%s_test.csv"%(DATASET),index_col=0)
target=data_df.iloc[:,-1]
feature=pd.DataFrame(data_df.iloc[:,:-1].values,dtype='float')
m,n=feature.shape
print(m,n)
print(feature.head())
print("Number of classes - ")
classes = np.unique(target)
for x in classes:
  print("Class -",x,"Number of Samples -", len(np.where(target == x)[0]))

feature_norm=pd.DataFrame(MinMaxScaler().fit_transform(feature))

In [ ]:
"""
Loading the Cluster JSON files from memory
"""

with open('%s_p%s_q%smi_cluster.json'%(DATASET, p, q), 'r') as fp:
  mi_cluster=json.load(fp)

with open('%s_p%s_q%srelief_cluster.json'%(DATASET, p, q), 'r') as fp:
  chi_cluster=json.load(fp)


with open('%s_p%s_q%schi_cluster.json'%(DATASET, p, q), 'r') as fp:
  relief_cluster=json.load(fp)

with open('%s_p%s_q%spc_cluster.json'%(DATASET, p, q), 'r') as fp:
  pc_cluster=json.load(fp)

with open('%s_p%s_q%sfs_cluster.json'%(DATASET, p, q), 'r') as fp:
  fs_cluster=json.load(fp)

with open('%s_p%s_q%stt_cluster.json'%(DATASET, p, q), 'r') as fp:
  tt_cluster=json.load(fp)

with open('%s_p%s_q%ssnr_cluster.json'%(DATASET, p, q), 'r') as fp:
  snr_cluster=json.load(fp)

In [ ]:
gene_repre_1 = get_augmented(feature, target, mi_cluster)
gene_repre_2 = get_augmented(feature, target, relief_cluster)
gene_repre_3 = get_augmented(feature, target, chi_cluster)
gene_repre_4 = get_augmented(feature, target, pc_cluster)
gene_repre_5 = get_augmented(feature, target, fs_cluster)
gene_repre_6 = get_augmented(feature, target, tt_cluster)
gene_repre_7 = get_augmented(feature, target, snr_cluster)

In [ ]:
gene_repre_1 = pd.DataFrame(gene_repre_1)
gene_repre_2 = pd.DataFrame(gene_repre_2)
gene_repre_3 = pd.DataFrame(gene_repre_3)
gene_repre_4 = pd.DataFrame(gene_repre_4)
gene_repre_5 = pd.DataFrame(gene_repre_5)
gene_repre_6 = pd.DataFrame(gene_repre_6)
gene_repre_7 = pd.DataFrame(gene_repre_7)

In [ ]:
#Can Skip this cell
print(gene_repre_1.shape)
print(gene_repre_2.shape)
print(gene_repre_3.shape)
print(gene_repre_4.shape)
print(gene_repre_5.head())
print(gene_repre_6.head())
print(gene_repre_7.head())

In [ ]:
def sort_keys(scores,gene_repre,target,flag=True):
  score_dict={}
  x=0
  for i in gene_repre.columns:
    score_dict[i]=scores[x]
    x+=1
  return [k for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)]

In [ ]:
"""
feature_ranking cannot be used here because it sorts and returns the indices from 0-1
They need to be sorted using a different function
"""
sorted_mi_keys=sort_keys(mutual_info_wrapper(gene_repre_1,target),gene_repre_1,target,True)[:q]

sorted_relief_keys=sort_keys(reliefF(gene_repre_2,target,k=NEIGHBOURS,repetitions=5),gene_repre_2,target,True)[:q]


mms=MinMaxScaler()
nfeature=mms.fit_transform(gene_repre_3)
chi_score,p_val=chi2(nfeature,target)
sorted_chi_keys=sort_keys(chi_score,gene_repre_3,target,False)[:q]

sorted_pc_keys=sort_keys(pearson_corr(gene_repre_4,target),gene_repre_4,target,True)[:q]
sorted_fs_keys=sort_keys(fisher_score(gene_repre_5.values,target),gene_repre_5,target,True)[:q]

sorted_tt_keys=sort_keys(t_test(gene_repre_6,target),gene_repre_6,target,True)[:q]

sorted_snr_keys = sort_keys(signaltonoise(gene_repre_7, target), gene_repre_7, target, True)[:q]

In [ ]:
print("MI cluster after sorting - ",sorted_mi_keys)
print("Relief cluster after sorting - ",sorted_relief_keys)
print("Chi cluster after sorting - ",sorted_chi_keys)
print("Pearson cluster after sorting - ",sorted_pc_keys)
print("Fisher cluster after sorting - ",sorted_fs_keys)
print("T-Test cluster after sorting - ",sorted_tt_keys)
print("SNR cluster after sorting - ",sorted_snr_keys)

In [ ]:
LOOCV=LeaveOneOut()
data_KNN = KNeighborsClassifier(n_neighbors= int(feature.shape[0] ** 0.5))
data_SVM = SVC(kernel='rbf',gamma='scale')
data_NB = GaussianNB()
data_Tree = DecisionTreeClassifier()
rows = feature.shape[0]

classifiers=["NB","KNN","Tree","SVM"]
classifier_dict = { "NB": GaussianNB(), 
                   "KNN": KNeighborsClassifier(n_neighbors= int(feature.shape[0] ** 0.5)), 
                   "Tree": DecisionTreeClassifier(), 
                   "SVM": SVC(kernel='rbf',gamma='scale')}

"""

"""

keys_list={"MI":sorted_mi_keys, "ReliefF":sorted_relief_keys, 
           "Chi":sorted_chi_keys,  
           "Fisher":sorted_fs_keys, "Pearson":sorted_pc_keys, "tTest":sorted_tt_keys, 
           "SNR":sorted_snr_keys}
"""
   
"""
cluster_list={"MI":gene_repre_1, "ReliefF":gene_repre_2,
              "Chi":gene_repre_3, "Pearson":gene_repre_4, 
              "Fisher":gene_repre_5,"tTest":gene_repre_6, 
              "SNR":gene_repre_7}

In [ ]:
acc_matrix = pd.DataFrame()
for i in range(1,q+1):
  """
  Make a dataframe out of i keys from each gene_representatives from their 
  respective sorted keys. 
  Than use LOOCV to measure accuracy on Train Dataset.
  """
  cluster_df=pd.DataFrame()
  s=0
  for filter_name in keys_list:
    tem_df = cluster_list[filter_name][keys_list[filter_name][:i]]
    for x in range(i):
      cluster_df[s] = tem_df.iloc[:, x]
      s += 1
  # print(cluster_df.shape)
  acc=0
  individual_acc = np.zeros(4)
  
  for train_index,test_index in LOOCV.split(cluster_df):
    """
    Data is divided into train-test splits and then polling method is used 
    to find the classification results (ensemble of KNN,SVM,NB,Decision Tree)
    """
    train_data,train_labels=cluster_df.iloc[train_index,:],target[train_index]
    test_data,test_labels=cluster_df.iloc[test_index,:],target[test_index].values.tolist()[0]
    data_KNN.fit(train_data,train_labels)
    data_SVM.fit(train_data,train_labels)
    data_NB.fit(train_data,train_labels)
    data_Tree.fit(train_data,train_labels)

    class_list = [data_NB, data_KNN, data_Tree, data_SVM]
    results=[]
    for x in range(4):
      tem_result = class_list[x].predict(test_data)[0]
      if tem_result == test_labels:
        individual_acc[x]+=1
      results.append(tem_result)
    polling_result=0
    max_freq=0
    for x in results:
      freq=results.count(x)
      if freq>max_freq:
        max_freq=freq
        polling_result=x
    if polling_result == test_labels:
      acc+=1
  individual_acc = np.round(individual_acc/cluster_df.shape[0],4)
  individual_acc = np.append(individual_acc, np.round(acc/cluster_df.shape[0],4))
  # print(individual_acc)
  acc_matrix[i] = individual_acc
acc_matrix = acc_matrix.T
acc_matrix.columns = classifiers[:]+['Ensemble']
print(acc_matrix)
acc_matrix.to_csv("Test-Multi-SGC-%s_p%s_q%s_Accuracy_Matrix.csv"%(DATASET, p, q))

In [ ]:
acc_df = []
for i in range(1, q+1):  
  #Ensemble of same type classifiers for each filter

  #records the ensemble classifier accuracy of each classifier with i genes
  classifier_accuracy = [] 

  for each_classifier in classifier_dict:
    
    #records the accuracy of the current classifier
    acc=0
    for train_index, test_index in LOOCV.split(range(rows)):
      """
      Data is divided into train-test splits and then polling method is used 
      to find the classification results (ensemble of KNN,SVM,NB,Decision Tree)
      """
      
      filter_result = []
      for filter_name in keys_list:

        #taking the train-test split from each filter
        train_data,train_labels = cluster_list[filter_name].iloc[train_index,:i],target[train_index]
        test_data,test_labels = cluster_list[filter_name].iloc[test_index,:i],target[test_index].values.tolist()[0]

        #filter wise fit and predict from the same type of classifier
        classifier_dict[each_classifier].fit(train_data, train_labels)
        filter_result.append(classifier_dict[each_classifier].predict(test_data)[0])

      #generating the polling result of all filters' classifiers
      polling_result = None
      max_freq = 0
      for each in set(filter_result):
        freq = filter_result.count(each)
        if freq>max_freq:
          max_freq = freq
          polling_result = each

      if polling_result == test_labels:
        acc+=1
    
    acc = np.round(acc/rows,4)
    classifier_accuracy.append(acc)

  #records the ensemble classifier accuracy from 1 to q genes for each classifier
  acc_df.append(classifier_accuracy)    
acc_df = pd.DataFrame(acc_df, columns= classifier_dict.keys())
acc_df.to_csv("Test-EC-Multi-SGC-%s_p%s_q%s_Accuracy_Matrix.csv"%(DATASET, p, q))
print(acc_df)

In [ ]:
files.download("Test-EC-Multi-SGC-%s_p%s_q%s_Accuracy_Matrix.csv"%(DATASET, p, q))

In [ ]:
files.download("Test-Multi-SGC-%s_p%s_q%s_Accuracy_Matrix.csv"%(DATASET, p, q))